### For Enterprise / System level code

In [ ]:
# importing libraries
import requests
import json
import time
import re
import pandas as pd
from datetime import date
import numpy as np
import base64

class orangesGamesAPI:

    def __init__(self):
        """
        Define base parameters
        """
        self.label_id= uuuu
        self.authorization_token= xxxx
        self.account_token= yyyy
        self.base_url="https://secure.orangesgames.com/api/"
    
    # Exponential Backoff algorithm to get away with issue of error 429
    @staticmethod
    def make_get_request_with_backoff(url:str, headers:str, params:str, payload:str,  number_of_retry:int):
        for counter in range(number_of_retry):
            try:
                response = requests.request("GET", url, headers=headers, params=params, data=payload)
                if response.status_code == 429:
                    wait_time= 2** counter
                    time.sleep(wait_time)
                else:
                    return response
            except Exception as error:
                print(f"Error:/n{error}")
                return None
    
    def make_request(self, Type: str, url_extension: str, data: dict = None):
        """
        Make GET/POST call to Orange Game API
        """
        if Type== "GET":

            # condition to handle API type ( aggregate list level or individial Id)
            api_type= url_extension.split("/")[-1]
            # Attempt to convert api_type to an integer
            try:
                api_type = int(api_type)  # Convert to integer
                print(f"work_id: {api_type}")
                # Handling GET requests (List level)
                if isinstance(api_type, int) and Type == "GET":
                    if data is None:
                        payload = {}
                    else:
                        payload = data

                    headers = {
                        'Accept': 'application/json',
                        'Authorization': f"Token {self.authorization_token}",
                        'Account-Token': self.account_token,
                        'X-Api-Version': '2024-06-30'
                    }
                    params = None
                    url = self.base_url + url_extension
                    
                    response = self.make_get_request_with_backoff(url=url, 
                                                                 headers=headers, 
                                                                 params=params, 
                                                                 payload=payload,  
                                                                 number_of_retry=5)

                    if response is None:
                        raise ValueError("No response received from the API.")
                        pass
                    else:
                        # print(f"Response Status: {response.status_code}")
                        data_dump = json.loads(response.content)

                        return data_dump

            except ValueError:
                # Handling GET requests ( List level )
                if isinstance(api_type, str) and Type == "GET":
                    if data is None:
                        payload = {}
                    else:
                        payload = data

                    headers = {
                        'Accept': 'application/json',
                        'Authorization': f"Token {self.authorization_token}",
                        'Account-Token': self.account_token,
                        'X-Api-Version': '2024-06-30'
                    }
                    params = {'per_page': 100}  # Set maximum records per page to 100
                    url = self.base_url + url_extension
                    start_cursor = None
                    page_count = 0

                    data_dump = []
                    while True:
                        try:
                            if start_cursor:
                                params['start_cursor'] = start_cursor

                            # Make the API request
                            response = self.make_get_request_with_backoff(
                                url=url,
                                headers=headers,
                                params=params,
                                payload=payload,
                                number_of_retry=5
                            )

                            # Check for no response
                            if response is None:
                                print("Warning: No response received, retrying...")
                                time.sleep(2)  # Retry after a small delay
                                continue

                            # Process response
                            print(f"Response Status: {response.status_code}")
                            if response.status_code != 200:
                                print(f"API returned error: {response.status_code} - {response.text}")
                                time.sleep(2)  # Retry after delay for non-200 responses
                                continue

                            data = response.json()

                            # Extract records and next_cursor
                            results = data.get('records', [])
                            start_cursor = data.get('next_cursor')

                            # Append data to data_dump
                            if results:
                                data_dump.extend(results)

                            page_count += 1
                            print(f"Page number: {page_count}, Records fetched: {len(results)}")

                            # Apply delay for every 100 pages
                            if page_count % 100 == 0:
                                print("Applying delay after 100 pages...")
                                time.sleep(15)

                            # Break the loop if no more data
                            if not start_cursor:
                                print("No more pages to fetch. Exiting.")
                                break

                        except Exception as err:
                            print(f"Unexpected Error:\n{err}")
                            # Optional: log or save partial data_dump before returning
                            return data_dump

                    return data_dump
    
        # Handling POST requests if needed
        elif Type == "POST":
                headers = {
                    'Accept': 'application/json',
                    'Authorization': f"Token {self.authorization_token}",
                    'Account-Token': self.account_token,
                    'Content-Type': 'application/json'
                }

                response = requests.request("POST", self.base_url + url_extension, headers=headers, json=data)
                print(f"Response Status: {response.status_code}")
                return json.loads(response.content)
        else:
            raise ValueError("Type must be 'GET' or 'POST'")
    
    def create_id_dump(self, main_api, obj_id):
        # Fetch data for the specified id such as work orders/ issues.
        data_dump = self.make_request(Type="GET", 
                                      url_extension=f"/v2/{main_api}/{obj_id}", 
                                      data=None)
        return data_dump